In [13]:
import csv
from itertools import groupby
import statistics
import timeit
import sys

In [132]:
with open('/global/home/users/pierrj/testfiles/500.confirmed', newline = '') as confirmed:
    confirmed_reader = csv.reader(confirmed, delimiter = '\t')
    confirmed_list = [[int(row[0]), int(row[1]), int(row[2])] for row in confirmed_reader]

In [102]:
with open('/global/scratch/users/pierrj/eccDNA/magnaporthe_pureculture/rawdata/illumina/G3_1A_tests/parallel.confirmed') as confirmedall:
    confirmedall_reader = csv.reader(confirmedall, delimiter = '\t')
    confirmedall_list = [[int(row[0]), int(row[1]), int(row[2])] for row in confirmedall_reader]

In [114]:
with open('/global/scratch/users/pierrj/eccDNA/magnaporthe_pureculture/rawdata/illumina/G3_1A_tests/genomecoverage.mergedandpe.G3_1A_bwamem.bed') as coverage:
    coverage_reader = csv.reader(coverage, delimiter = '\t')
    coverage_indexed = [[] for i in range(56)]
    for row in coverage_reader:
        coverage_indexed[(int(row[0][10:12])-1)].append([int(row[1]) -1, int(row[2])])

In [14]:
def get_size(obj, seen=None):

    """Recursively finds size of objects"""

    size = sys.getsizeof(obj)

    if seen is None:

        seen = set()

    obj_id = id(obj)

    if obj_id in seen:

        return 0

    # Important mark as seen *before* entering recursion to gracefully handle

    # self-referential objects

    seen.add(obj_id)

    if isinstance(obj, dict):

        size += sum([get_size(v, seen) for v in obj.values()])

        size += sum([get_size(k, seen) for k in obj.keys()])

    elif hasattr(obj, '__dict__'):

        size += get_size(obj.__dict__, seen)

    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):

        size += sum([get_size(i, seen) for i in obj])

    return size

In [115]:
print(get_size(coverage_indexed))

5071263964


In [111]:
def checkcoverage(ecc):
    region_len = ecc[2] - ecc[1]
    region = coverage_indexed[ecc[0]][ecc[1]:(ecc[2]+1)]
    region_cov = [region[i][1] for i in range(len(region))]
    beforestart = ecc[1] - 1 - region_len
    if beforestart <= 0:
        return 'region too close to start'
    afterstart = ecc[2] + 1
    if afterstart + region_len > coverage_indexed[ecc[0]][-1][0]:
        return 'region too close to end'
    region_before = coverage_indexed[ecc[0]][beforestart:beforestart + 1 + region_len]
    region_before_cov = [region_before[i][1] for i in range(len(region_before))]
    region_after = coverage_indexed[ecc[0]][afterstart:afterstart + 1 + region_len]
    region_after_cov = [region_after[i][1] for i in range(len(region_after))]
    mean_region = statistics.mean(region_cov)
    mean_before = statistics.mean(region_before_cov)
    mean_after = statistics.mean(region_after_cov)
    if mean_region >= (2*mean_before) and mean_region >= (2*mean_after):
        return 'true'
    else:
        return 'coverage too low'

In [133]:
for i in range(len(confirmed_list)):
    ecc = confirmed_list[i]
    region_len = ecc[2] - ecc[1]
    region = coverage_indexed[ecc[0]][ecc[1]:(ecc[2]+1)]
    region_cov = [region[i][1] for i in range(len(region))]
    beforestart = ecc[1] - 1 - region_len
    if beforestart <= 0:
        ecc.append('lconf')
        ecc.append('too close to start')
        print(ecc)
        continue
    afterstart = ecc[2] + 1
    if afterstart + region_len > coverage_indexed[ecc[0]][-1][0]:
        ecc.append('lconf')
        ecc.append('too close to end')
        print(ecc)
        continue
    region_before = coverage_indexed[ecc[0]][beforestart:beforestart + 1 + region_len]
    region_before_cov = [region_before[i][1] for i in range(len(region_before))]
    region_after = coverage_indexed[ecc[0]][afterstart:afterstart + 1 + region_len]
    region_after_cov = [region_after[i][1] for i in range(len(region_after))]
    mean_region = statistics.mean(region_cov)
    mean_before = statistics.mean(region_before_cov)
    mean_after = statistics.mean(region_after_cov)
    if mean_region >= (2*mean_before) and mean_region >= (2*mean_after):
        ecc.append('hconf')
        ecc.append('none')
    else:
        ecc.append('lconf')
        ecc.append('coverage too low')
    print(ecc)

[14, 70735, 71303, 'hconf', 'none']
[14, 349128, 349764, 'hconf', 'none']
[2, 1674894, 1680235, 'lconf', 'coverage too low']
[12, 355104, 356042, 'lconf', 'coverage too low']
[2, 626071, 627316, 'hconf', 'none']
[21, 209465, 209961, 'lconf', 'coverage too low']
[4, 2696604, 2699034, 'lconf', 'coverage too low']
[14, 132347, 132906, 'lconf', 'coverage too low']
[9, 932917, 935653, 'lconf', 'coverage too low']
[5, 1887602, 1889159, 'hconf', 'none']
[8, 166497, 166880, 'hconf', 'none']
[4, 3034338, 3034647, 'lconf', 'coverage too low']
[1, 4945547, 4946469, 'lconf', 'coverage too low']
[2, 573874, 575341, 'lconf', 'coverage too low']
[11, 748313, 748925, 'lconf', 'coverage too low']
[2, 3611761, 3616320, 'hconf', 'none']
[7, 1295634, 1297723, 'lconf', 'coverage too low']
[2, 3923725, 3924836, 'lconf', 'coverage too low']
[4, 1884379, 1885179, 'lconf', 'coverage too low']
[3, 1200621, 1200838, 'hconf', 'none']
[8, 1087179, 1087434, 'hconf', 'none']
[3, 3985805, 3986558, 'hconf', 'none']
[1

In [129]:
hconf_lowconf = list(map(checkcoverage, confirmed_list))

In [130]:
print(hconf_lowconf)

['true', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'true', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'true', 'region too close to start', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'true', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too 

In [100]:
start_time = timeit.default_timer()
hconf_lowconf = list(map(checkcoverage, confirmed_list))
print(timeit.default_timer() - start_time)

0.8786546527408063


In [118]:
start_time = timeit.default_timer()
hconf_lowconf_all = list(map(checkcoverage, confirmedall_list))
print(timeit.default_timer() - start_time)

1350.8100269818678


In [119]:
print(hconf_lowconf_all)

['true', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'true', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'true', 'region too close to start', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'true', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'true', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'coverage too low', 'true', 'coverage too low', 'c